In [65]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import datetime

Open the conection with the NFL website

In [66]:
connection = urlopen('http://www.nfl.com/stats/categorystats?archive=false&conference=null&statisticCategory=PASSING&season=2018&seasonType=REG&experience=&tabSeq=0&qualified=false&Submit=Go')

bs = BeautifulSoup(connection, 'html.parser')

In [87]:
def getNflData(attribute, season_type):
    
    data = pd.DataFrame()
    try:
        attribute = attribute.upper()
        
    except AttributeError:
        print('Please insert a valid attribute such as "Passing", "Rushing", etc')
        return('AttributeError')
    
    try:
        season_type = season_type.upper()
        
    except AttributeError:
        print('Please insert a valid season type such as "REG" for regular season and "POST" for postseason')
        return('AttributeError')
      
    actual_year = (datetime.datetime.today()).year
    #actual_year = 1973
    for year in range(1966, actual_year ): 
        try:
            connection = urlopen(f'http://www.nfl.com/stats/categorystats?archive=false&conference=null&statisticCategory={attribute}&season={year}&seasonType={season_type}&experience=&tabSeq=0&qualified=false&Submit=Go')
        except HTTPError:
            print(f'The season of {year} was not collected')
            pass
        
        bs = BeautifulSoup(connection, 'html.parser')
    
        players_info = [re.sub('(\\n|\\t)', ',', col.get_text()) for col in bs.find_all('tr')]
    
        players_info = [re.sub(',{2,}', ', ', player) for player in players_info]

        players_info = [re.sub(r'([0-9]{1}),([0-9]{1})', r'\1.\2', player) for player in players_info]

        players_info = [re.sub(', ', ',', player) for player in players_info]

        players_info = [re.sub('(^,|,$)', '', player) for player in players_info]
        
        players = [players.split(',') for players in players_info]

        players = pd.DataFrame(data = players[1:], columns = players[0])
        
        #print(players)

        players['Year'] = year
        
        data = data.append(players)
    
    return data


In [89]:
x = getNflData('passing', 'REG')

x.to_csv(r'nfl.csv')

,Rk,Player,Team,Pos,Comp,Att,Pct,Att/G,Yds,Avg,...,TD,Int,1st,1st%,Lng,20+,40+,Sck,Rate,Year
0,1,Sonny Jurgensen,PHI,QB,196,366,53.6,26.1,3.261,8.9,...,22,26,0,0.0,--,0,0,0,74.3,1962
1,2,Y.A. Tittle,NYG,QB,200,375,53.3,26.8,3.224,8.6,...,33,20,0,0.0,--,0,0,0,89.5,1962
2,3,Billy Wade,CHI,QB,225,412,54.6,29.4,3.172,7.7,...,18,24,0,0.0,--,0,0,0,70.0,1962
3,4,Johnny Unitas,BAL,QB,222,389,57.1,27.8,2.967,7.6,...,23,23,0,0.0,--,0,0,0,76.5,1962
4,5,Norm Snead,WAS,QB,184,354,52.0,25.3,2.926,8.3,...,22,22,0,0.0,--,0,0,0,74.7,1962
5,6,Frank Tripucka,DEN,QB,240,440,54.5,31.4,2.917,6.6,...,17,25,0,0.0,--,0,0,0,64.4,1962
6,7,George Blanda,HOU,QB,197,418,47.1,29.9,2.810,6.7,...,27,42,0,0.0,--,0,0,0,51.3,1962
7,8,Len Dawson,TEX,QB,189,310,61.0,22.1,2.759,8.9,...,29,17,0,0.0,--,0,0,0,98.3,1962
8,9,Fran Tarkenton,MIN,QB,163,329,49.5,23.5,2.595,7.9,...,22,25,0,0.0,--,0,0,0,66.9,1962
9,10,Charley Johnson,STL,QB,150,308,48.7,28.0,2.440,7.9,...,16,20,0,0.0,--,0,0,0,65.9,1962


In [46]:

players_info = [re.sub('(\\n|\\t)', ',', col.get_text()) for col in bs.find_all('tr')]

players_info = [re.sub(',{2,}', ', ', player) for player in players_info]

players_info = [re.sub(r'([0-9]{1}),([0-9]{1})', r'\1.\2', player) for player in players_info]

players_info = [re.sub(', ', ',', player) for player in players_info]

players_info = [re.sub('(^,|,$)', '', player) for player in players_info]

players_info



['Rk,Player,Team,Pos,Comp,Att,Pct,Att/G,Yds,Avg,Yds/G,TD,Int,1st,1st%,Lng,20+,40+,Sck,Rate',
 '1,Ben Roethlisberger,PIT,QB,452,675,67.0,42.2,5.129,7.6,320.6,34,16,248,36.7,97T,61,16,24,96.5',
 '2,Patrick Mahomes,KC,QB,383,580,66.0,36.2,5.097,8.8,318.6,50,12,237,40.9,89T,75,15,26,113.8',
 '3,Matt Ryan,ATL,QB,422,608,69.4,38.0,4.924,8.1,307.8,35,7,236,38.8,75T,56,9,42,108.1',
 '4,Jared Goff,LA,QB,364,561,64.9,35.1,4.688,8.4,293.0,32,12,233,41.5,70T,69,9,33,101.1',
 '5,Andrew Luck,IND,QB,430,639,67.3,39.9,4.593,7.2,287.1,39,15,236,36.9,68T,53,7,18,98.7',
 '6,Aaron Rodgers,GB,QB,372,597,62.3,37.3,4.442,7.4,277.6,25,2,200,33.5,75T,55,16,49,97.6',
 '7,Tom Brady,NE,QB,375,570,65.8,35.6,4.355,7.6,272.2,29,11,205,36.0,63T,53,8,21,97.7',
 '8,Philip Rivers,LAC,QB,347,508,68.3,31.8,4.308,8.5,269.2,32,12,213,41.9,75T,60,10,32,105.5',
 '9,Eli Manning,NYG,QB,380,576,66.0,36.0,4.299,7.5,268.7,21,11,206,35.8,58,57,10,47,92.4',
 '10,Kirk Cousins,MIN,QB,425,606,70.1,37.9,4.298,7.1,268.6,30,10,218,36.0,75

In [51]:
players = [players.split(',') for players in players_info]

players = pd.DataFrame(data = players[1:], columns = players[0])

#players


x = pd.DataFrame()

x = x.append(players)

print(x) 

    Rk              Player Team Pos Comp  Att   Pct Att/G    Yds  Avg  Yds/G  \
0    1  Ben Roethlisberger  PIT  QB  452  675  67.0  42.2  5.129  7.6  320.6   
1    2     Patrick Mahomes   KC  QB  383  580  66.0  36.2  5.097  8.8  318.6   
2    3           Matt Ryan  ATL  QB  422  608  69.4  38.0  4.924  8.1  307.8   
3    4          Jared Goff   LA  QB  364  561  64.9  35.1  4.688  8.4  293.0   
4    5         Andrew Luck  IND  QB  430  639  67.3  39.9  4.593  7.2  287.1   
5    6       Aaron Rodgers   GB  QB  372  597  62.3  37.3  4.442  7.4  277.6   
6    7           Tom Brady   NE  QB  375  570  65.8  35.6  4.355  7.6  272.2   
7    8       Philip Rivers  LAC  QB  347  508  68.3  31.8  4.308  8.5  269.2   
8    9         Eli Manning  NYG  QB  380  576  66.0  36.0  4.299  7.5  268.7   
9   10        Kirk Cousins  MIN  QB  425  606  70.1  37.9  4.298  7.1  268.6   
10  11      Deshaun Watson  HOU  QB  345  505  68.3  31.6  4.165  8.2  260.3   
11  12          Derek Carr  OAK  QB  381

In [32]:
x = datetime.timedelta(weeks = 52)
d = datetime.datetime.today() - datetime.timedelta(days=365)
actual_year = (datetime.datetime.today()).year

In [33]:
d.year
actual_year

2019